JuMP has a Direct, an Automatic and a Manual mode. In this tutorial, we look at the difference between these modes.

We illustrate this different modes using the CSDP interface. CSDP supports problems in the following format
$$
\begin{align*}
\text{maximize} \quad \langle C, X\rangle\\
    \langle A_i, X\rangle & = b_i\\
          X & \succeq 0
\end{align*}
$$
where `X` is a block-diagonal matrix.

The problem is inputed as follows:
1. Provide number of linear constraints and list of blocks size.
2. Provide the list of nonzero entries of `C`, `A_i` and values of `b_i`.

Consider the problem of finding the maximum $\alpha$ such that $x^2 + \alpha x + 1$ is nonnegative for all $x \in \mathbb{R}$.
It can be written as an SDP as follows:
$$
\begin{align*}
\text{maximize} \quad 2X_{1, 2}\\
    X_{1, 1} & = 1\\
    X_{2, 2} & = 1\\
    X & \succeq 0
\end{align*}
$$
The CSDP form is
$$
\begin{align*}
\text{maximize} \quad \langle \begin{pmatrix}0 & 1\\1 & 0\end{pmatrix}, X\rangle\\
    \langle \begin{pmatrix}1 & 0\\0 & 0\end{pmatrix}, X\rangle & = 1\\
    \langle \begin{pmatrix}0 & 0\\0 & 1\end{pmatrix}, X\rangle & = 1\\
          X & \succeq 0
\end{align*}
$$

## Caching Optimizer

The CSDP interface does not support loading the problem incrementally as it requires having the list of blocks and number of constraints before loading any constraint data.
Therefore, adding variables or constraints through `MOI.add_variable` and `MOI.add_constraint` is not supported.

In [79]:
import CSDP
optimizer = CSDP.Optimizer()
using JuMP
model = direct_model(optimizer)
@variable(model, α)

MathOptInterface.AddVariableNotAllowed: MathOptInterface.AddVariableNotAllowed: Adding variables cannot be performed. You may want to use a `CachingOptimizer` in `AUTOMATIC` mode or you may need to call `reset_optimizer` before doing this operation if the `CachingOptimizer` is in `MANUAL` mode.

The problem needs to be cached and loaded at once so that the CSDP MOI wrapper can read the whole problem, initialize CSDP with the number of constraints and list of blocks, and then load the coefficients into `C`, `A_i` and `b_i`.

In [80]:
cached = MOIU.CachingOptimizer(JuMP._MOIModel{Float64}(), optimizer)
model = direct_model(cached)
show(model)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CSDP

As we can see below, JuMP believes we have created the model in `AUTOMATIC` mode because we have added a cache.

In [81]:
@variable(model, X[1:2, 1:2], PSD)

2×2 LinearAlgebra.Symmetric{VariableRef,Array{VariableRef,2}}:
 X[1,1]  X[1,2]
 X[1,2]  X[2,2]

In [82]:
show(model)

A JuMP Model
Feasibility problem with:
Variables: 3
`Array{VariableRef,1}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CSDP
Names registered in the model: X

We can see above that the model was detached because the CSDP optimizer does not support `add_constraint`, the model will be copied all at once at `optimize!`.

In [83]:
@constraint(model, X[1, 1] == 1)

X[1,1] = 1.0

In [84]:
@constraint(model, X[2, 2] == 1)

X[2,2] = 1.0

In [85]:
@objective(model, Max, 2X[1, 2])

2 X[1,2]

In [86]:
optimize!(model)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj:  2.3431458e-01 Ad: 1.00e+00 Dobj:  3.2435027e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  3.5817600e-01 Ad: 1.00e+00 Dobj:  1.6458531e+01 
Iter:  3 Ap: 1.00e+00 Pobj:  6.3421738e-01 Ad: 1.00e+00 Dobj:  8.6843940e+00 
Iter:  4 Ap: 1.00e+00 Pobj:  1.0638920e+00 Ad: 1.00e+00 Dobj:  5.0889793e+00 
Iter:  5 Ap: 1.00e+00 Pobj:  1.4516765e+00 Ad: 1.00e+00 Dobj:  3.4642192e+00 
Iter:  6 Ap: 1.00e+00 Pobj:  1.6669906e+00 Ad: 1.00e+00 Dobj:  2.6732609e+00 
Iter:  7 Ap: 1.00e+00 Pobj:  1.8020927e+00 Ad: 1.00e+00 Dobj:  2.3052268e+00 
Iter:  8 Ap: 1.00e+00 Pobj:  1.8891872e+00 Ad: 1.00e+00 Dobj:  2.1407533e+00 
Iter:  9 Ap: 1.00e+00 Pobj:  1.9410015e+00 Ad: 1.00e+00 Dobj:  2.0667836e+00 
Iter: 10 Ap: 1.00e+00 Pobj:  1.9999978e+00 Ad: 9.00e-01 Dobj:  2.0066746e+00 
Iter: 11 Ap: 1.00e+00 Pobj:  2.0000000e+00 Ad: 9.00e-01 Dobj:  2.0006657e+00 
Iter: 12 Ap: 1.00e+00 Pobj:  1.9998331e+00 Ad: 1.00e+

In [87]:
show(model)

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`Array{VariableRef,1}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CSDP
Names registered in the model: X

As we can see above, CSDP has been attached in just before calling `MOI.optimize!`.

In [88]:
value.(X)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

In [89]:
objective_value(model)

1.9999999791293397

## Constraint Bridges

Now suppose we would like to maximize $\alpha$ such that $\beta x^2 + \alpha x + \gamma$ is nonnegative for all $x \in \mathbb{R}$ and $\beta + \gamma \le 2$.

In [90]:
MOI.empty!(cached)
model = direct_model(cached)
@variable(model, X[1:2, 1:2], PSD)
@constraint(model, X[1, 1] + X[2, 2] ≤ 2)

ErrorException: Constraints of type MathOptInterface.ScalarAffineFunction{Float64}-in-MathOptInterface.LessThan{Float64} are not supported by the solver, try using `bridge_constraints=true` in the `JuMP.Model` constructor if you believe the constraint can be reformulated to constraints supported by the solver.

As pointed out by the error message, the solver does not support inequality between scalar affine function.
However, as it supports equality between affine functions and nonnegative variables (1x1 blocks in the `X` matrix), it could reformulate it into an equivalent form supported by the solver by adding a slack variable.
This is achieved by the `ScalarSlackBridge`.

In [91]:
MOI.empty!(cached)
bridged = MOI.Bridges.full_bridge_optimizer(cached, Float64)
model = direct_model(bridged)
@variable(model, X[1:2, 1:2], PSD)
@constraint(model, X[1, 1] + X[2, 2] ≤ 2)

X[1,1] + X[2,2] ≤ 2.0

In [92]:
@objective(model, Max, 2X[1, 2])
optimize!(model)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj:  1.6905989e-01 Ad: 1.00e+00 Dobj:  3.0908574e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  3.4998523e-01 Ad: 9.00e-01 Dobj:  3.4058426e+00 
Iter:  3 Ap: 1.00e+00 Pobj:  1.6203778e+00 Ad: 1.00e+00 Dobj:  3.1483057e+00 
Iter:  4 Ap: 1.00e+00 Pobj:  1.6492373e+00 Ad: 1.00e+00 Dobj:  2.4132005e+00 
Iter:  5 Ap: 1.00e+00 Pobj:  1.7961213e+00 Ad: 1.00e+00 Dobj:  2.1781021e+00 
Iter:  6 Ap: 9.00e-01 Pobj:  1.9795801e+00 Ad: 9.00e-01 Dobj:  2.0177767e+00 
Iter:  7 Ap: 9.00e-01 Pobj:  1.9979580e+00 Ad: 1.00e+00 Dobj:  1.9999984e+00 
Iter:  8 Ap: 9.00e-01 Pobj:  1.9997958e+00 Ad: 1.00e+00 Dobj:  1.9999984e+00 
Iter:  9 Ap: 5.12e-01 Pobj:  1.9998934e+00 Ad: 1.00e+00 Dobj:  2.0000052e+00 
Iter: 10 Ap: 1.00e+00 Pobj:  1.9999995e+00 Ad: 1.00e+00 Dobj:  1.9999998e+00 
Iter: 11 Ap: 1.00e+00 Pobj:  2.0000000e+00 Ad: 1.00e+00 Dobj:  2.0000000e+00 


In [93]:
value.(X)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

## Model cache

Suppose now that we would like to solve the same problem with SDPA.

In [94]:
import SDPA
set_optimizer(model, with_optimizer(SDPA.Optimizer))

ErrorException: The `set_optimizer` function is not supported in DIRECT mode.

The `set_optimizer` function is not supported in DIRECT mode because changing the optimizer would invalidate the variables and constraint references hold by the user.
Moreover, we cannot just reset the optimizer in `cached` since SDPA might need different bridges, and changing the bridges might change the indices of constraints.
So resolve this issue, we use a cache of the model before it is bridged.

In [95]:
show(model)

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`Array{VariableRef,1}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: DIRECT
Solver name: CSDP
Names registered in the model: X

As we can see above, the model is considered to be in DIRECT mode because type of the backend is `LazyBridgeOptimizer`, not `CachingOptimizer`.

In [96]:
MOI.empty!(bridged)
backend = MOIU.CachingOptimizer(JuMP._MOIModel{Float64}(), MOIU.AUTOMATIC)
MOIU.reset_optimizer(backend, bridged)
model = direct_model(backend)
show(model)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CSDP

As can be seen above, JuMP believes we created the model in AUTOMATIC mode because we created a cache for the unbridged model in `MOIU.AUTOMATIC` mode.

In [97]:
@variable(model, X[1:2, 1:2], PSD)
cref = @constraint(model, X[1, 1] + X[2, 2] ≤ 2)
@objective(model, Max, 2X[1, 2])
show(model)

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`Array{VariableRef,1}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CSDP
Names registered in the model: X

As we can see above, the inequality between affine expressions is not bridged in the backend model (we may switch to an optimizer supporting this type of constraints!).

In [98]:
optimize!(model)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj:  1.6905989e-01 Ad: 1.00e+00 Dobj:  3.0908574e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  3.4998523e-01 Ad: 9.00e-01 Dobj:  3.4058426e+00 
Iter:  3 Ap: 1.00e+00 Pobj:  1.6203778e+00 Ad: 1.00e+00 Dobj:  3.1483057e+00 
Iter:  4 Ap: 1.00e+00 Pobj:  1.6492373e+00 Ad: 1.00e+00 Dobj:  2.4132005e+00 
Iter:  5 Ap: 1.00e+00 Pobj:  1.7961213e+00 Ad: 1.00e+00 Dobj:  2.1781021e+00 
Iter:  6 Ap: 9.00e-01 Pobj:  1.9795801e+00 Ad: 9.00e-01 Dobj:  2.0177767e+00 
Iter:  7 Ap: 9.00e-01 Pobj:  1.9979580e+00 Ad: 1.00e+00 Dobj:  1.9999984e+00 
Iter:  8 Ap: 9.00e-01 Pobj:  1.9997958e+00 Ad: 1.00e+00 Dobj:  1.9999984e+00 
Iter:  9 Ap: 5.12e-01 Pobj:  1.9998934e+00 Ad: 1.00e+00 Dobj:  2.0000052e+00 
Iter: 10 Ap: 1.00e+00 Pobj:  1.9999995e+00 Ad: 1.00e+00 Dobj:  1.9999998e+00 
Iter: 11 Ap: 1.00e+00 Pobj:  2.0000000e+00 Ad: 1.00e+00 Dobj:  2.0000000e+00 


In [99]:
set_optimizer(model, with_optimizer(SDPA.Optimizer))

In [100]:
optimize!(model)

Entering DMUMPS driver with JOB, N, NZ =  -2           0              0
Strange behavior : primal < dual :: line 144 in sdpa_solve.cpp


In [101]:
value.(X)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

## Automatic mode

In automatic mode, the caching containing the unbridged model is created in automatic mode, and bridges are automatically applied if `bridge_constraints` is `true`.
The cache containing the bridged model is only created if required, e.g. if the model implements `add_variable` and `add_constraint`, it does not need this cache.
There are currently three ways to implement copy:
* If the solver supports loading the problem incrementally, implement
  `add_variable`, `add_constraint` for supported constraints and
  [`set`](@ref) for supported attributes and add:
  ```julia
  function MOI.copy_to(dest::Optimizer, src::MOI.ModelLike; kws...)
      return MOI.Utilities.automatic_copy_to(dest, src; kws...)
  end
  ```
  with
  ```julia
  MOI.Utilities.supports_default_copy_to(model::Optimizer, copy_names::Bool) = true
  ```
  or
  ```julia
  MOI.Utilities.supports_default_copy_to(model::Optimizer, copy_names::Bool) = !copy_names
  ```
  depending on whether the solver support names.
  The copy will be carries out by `default_copy_to`.
* If the solver does not support loading the problem incrementally, we can either add a custom
  implementation of [`copy_to`](@ref);
* or implement the Allocate-Load API and do
  ```julia
  function MOI.copy_to(dest::Optimizer, src::MOI.ModelLike; kws...)
      return MOI.Utilities.automatic_copy_to(dest, src; kws...)
  end
  ```
  with
  ```julia
  MOI.Utilities.supports_allocate_load(model::Optimizer, copy_names::Bool) = true
  ```
  or
  ```julia
  MOI.Utilities.supports_allocate_load(model::Optimizer, copy_names::Bool) = !copy_names
  ```
  depending on whether the solver support names.

  Note that even if both writing a custom implementation of [`copy_to`](@ref)
  and implementing the [Allocate-Load API](@ref) requires the user to copy the
  model from a cache, the [Allocate-Load API](@ref) allows MOI layers to be
  added between the cache and the solver which allows transformations to be
  applied without the need for additional caching. For instance, with the
  proposed [Light bridges](https://github.com/JuliaOpt/MathOptInterface.jl/issues/523),
  no cache will be needed to store the bridged model when bridges are used by
  JuMP so implementing the [Allocate-Load API](@ref) will allow JuMP to use only
  one cache instead of two.

Currently, bridges only implement `add_constraint` and not the Allocate-Load API so a cache needs to be used for the bridged model in the two last cases.
However, it is planned to implement the Allocate-Load API for bridges (see [Light bridges](https://github.com/JuliaOpt/MathOptInterface.jl/issues/523)).
Then by implementing the Allocate-Load API, a solver would not need a cache for the bridged model.

To illustrate this, suppose we don't create the cache containing the bridged model.
As we can see below, `automatic_copy_to` does not find a method to copy the model from the cache:
* It cannot use `default_copy_to` as it is not supported by CSDP and
* it cannot use `allocate_load` because it is not supported by the bridges (this should be addressed by [Light bridges](https://github.com/JuliaOpt/MathOptInterface.jl/issues/523)).

In [102]:
MOI.empty!(optimizer)
bridged = MOI.Bridges.full_bridge_optimizer(optimizer, Float64)
backend = MOIU.CachingOptimizer(JuMP._MOIModel{Float64}(), MOIU.AUTOMATIC)
MOIU.reset_optimizer(backend, bridged)
model = direct_model(backend)
@variable(model, X[1:2, 1:2], PSD)
cref = @constraint(model, X[1, 1] + X[2, 2] ≤ 2)
@objective(model, Max, 2X[1, 2])
optimize!(model)

ErrorException: Model MathOptInterface.Bridges.LazyBridgeOptimizer{SemidefiniteOptInterface.SOItoMOIBridge{Float64,CSDP.SDOptimizer},MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}} does not support copy.

On the other hand, Mosek, supports incremental loading of the problem hence it can
* either be used with one cache for the unbridged model so that it is possible to change the optimizer (this is what is done in AUTOMATIC and MANUAL modes)
* or use no cache.

In [103]:
using MosekTools
optimizer = Mosek.Optimizer()
bridged = MOI.Bridges.full_bridge_optimizer(optimizer, Float64)
model = direct_model(bridged)
show(model)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: DIRECT
Solver name: Mosek

In [104]:
@variable(model, X[1:2, 1:2], PSD)
cref = @constraint(model, X[1, 1] + X[2, 2] ≤ 2)
@objective(model, Max, 2X[1, 2])
optimize!(model)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1               
  Cones                  : 0               
  Scalar variables       : 3               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

In [105]:
value.(X)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0